# Βήμα 1. Συλλογή δεδομένων:


In [4]:
import requests
from bs4 import BeautifulSoup
import json

#Συνάρτηση για να ανακτήσουμε συγκεκριμένα άρθρα
def fetch_articles(start_url, num_articles = 10):
    base_url = "https://en.wikipedia.org/"
    visited = set() #Σύνολο url τα οποία έχουμε ήδη επισκεφτεί
    articles = [] #Δεδομένα των άρθρων
    
    #Συνάρτηση για να συλλέξουμε τα δεδομένα της κάθε σελίδας
    def fetch_article_data(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.find('h1').text
        paragraphs = soup.find_all('p')
        content = " ".join([p.text for p in paragraphs])
        return {"title": title, "url": url, "content": content}
    
    #Τα url τα οποία θα επισκεφτούμε στην συνέχεια
    to_visit = [start_url]
    while to_visit and len(articles) < num_articles:
        current_url = to_visit.pop(0)
        if current_url in visited:
            continue
        visited.add(current_url)
        
        #Αποφεύγουμε το Main Page της Wikipedia
        if 'Main_Page' in current_url:
            continue
        
        print(f"Fetching: {current_url}")
        article = fetch_article_data(current_url)
        if article:
            articles.append(article)
        
        response = requests.get(current_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        for link in soup.find_all('a', href = True):
            href = link['href']
            if href.startswith('/wiki/') and ':' not in href:
                full_url = base_url + href
                if full_url not in visited:
                    to_visit.append(full_url)
        
    return articles
        

start_url = "https://en.wikipedia.org/wiki/Processor_(computing)"
articles = fetch_articles(start_url, num_articles = 9)

with open("articles.json", "w", encoding="utf-8") as file:
    json.dump(articles, file, ensure_ascii=False, indent=4)
print("The data has been saved to \"articles.json\"")

Fetching: https://en.wikipedia.org/wiki/Processor_(computing)
Fetching: https://en.wikipedia.org//wiki/Processor_(computing)
Fetching: https://en.wikipedia.org//wiki/Processor_(disambiguation)#Computing
Fetching: https://en.wikipedia.org//wiki/Computing
Fetching: https://en.wikipedia.org//wiki/Computer_science
Fetching: https://en.wikipedia.org//wiki/Circuit_(computer_science)
Fetching: https://en.wikipedia.org//wiki/Memory_(computing)
Fetching: https://en.wikipedia.org//wiki/Microprocessor
Fetching: https://en.wikipedia.org//wiki/Metal%E2%80%93oxide%E2%80%93semiconductor
The data has been saved to "articles.json"


# Βήμα 2. Προεπεξεργασία κειμένου (Text Processing):

In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Συνάρτηση για επεξεργασία κειμένου
def process_content(text):
    if not isinstance(text, str):  # Έλεγχος ότι η είσοδος είναι string
        return text
    
    # Tokenization
    words = word_tokenize(text)
    processed_words = []
    
    for word in words:
        if word.isdigit():  # Αν είναι αριθμός, το προσθέτουμε απευθείας
            processed_words.append(word)
        else:
            # Αφαίρεση ειδικών χαρακτήρων
            word = word.strip(string.punctuation)
            
            # Lemmatization
            lemmatizer = WordNetLemmatizer()
            word = lemmatizer.lemmatize(word)
            
            # Αφαίρεση stop words
            stop_words = set(stopwords.words('english'))
            if word.lower() not in stop_words:
                processed_words.append(word.lower())
    
    return ' '.join(processed_words)

# Ανάγνωση δεδομένων από αρχείο JSON
try:
    dataframe = pd.read_json("articles.json")  # διαδρομή αρχείου
except Exception as e:
    print(f"Error reading JSON file: {e}")
    exit()

# Επεξεργασία δεδομένων DataFrame
for column in dataframe.columns:
    dataframe[column] = dataframe[column].apply(process_content)

# Μετονομασία στηλών
new_columns = {
    dataframe.columns[0]: 'Title',
    dataframe.columns[1]: 'Url',
    dataframe.columns[2]: 'Content'
}
dataframe = dataframe.rename(columns=new_columns)

# Αποθήκευση επεξεργασμένων δεδομένων σε νέο αρχείο JSON
dataframe.to_json("processed.json", orient='records', indent=4)

print("The data has been saved to \"processed.json\"")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Μαρία\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Μαρία\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Μαρία\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


The data has been saved to "processed.json"


# Βήμα 3. Ευρετήριο (indexing): 

In [3]:
import json
from collections import defaultdict

# Διαβάζουμε τα επεξεργασμένα δεδομένα από το αρχείο processed.json
with open('processed.json', 'r') as json_file:
    documents = json.load(json_file)  # Φορτώνουμε το JSON σε μορφή λίστας/λεξικού

# Αντίστροφο ευρετήριο (inverted index)
inverted_index = defaultdict(list)  # Χρησιμοποιούμε defaultdict για να αποθηκεύσουμε τα IDs των εγγράφων

# Επεξεργασία κάθε έγγραφου
for doc_id, document in enumerate(documents):
    # Σπλιτάρω και αποθηκεύω στο terms όλα τα κομμάτια
    for field, field_value in document.items():
        if isinstance(field_value, str):  # Ελέγχω αν το πεδίο είναι string
            terms = field_value.split()  # Χωρίζω το κείμενο σε όρους
            
            # Δημιουργία του αντεστραμμένου ευρετηρίου (αντί για αριθμό εμφανίσεων, αποθηκεύουμε τα doc_id)
            for term in terms:
                if doc_id not in inverted_index[term]:  # Ελέγχουμε αν το doc_id είναι ήδη στη λίστα
                    inverted_index[term].append(doc_id)

# Αποθήκευση του αντεστραμμένου ευρετηρίου σε αρχείο JSON
with open('inverted_index.json', 'w') as json_file:
    json.dump(inverted_index, json_file, indent=4)  


print("The data has been saved to \"inverted_index.json\"")

The data has been saved to "inverted_index.json"


# Βήμα 4. Μηχανή αναζήτησης (Search Engine):

 α) Επεξεργασία ερωτήματος (Query Processing): 

In [ ]:
import json

# Φόρτωση ανεστραμμένου ευρετηρίου
def load_inverted_index(filepath):
    with open(filepath, "r") as f:
        return json.load(f)

# Boolean Search Function
def boolean_search(query, inverted_index):
    # Διαχωρισμός όρων και τελεστών
    terms = query.lower().split()  # Διαχωρισμός βάσει κενού
    if not terms:
        return set()  # Επιστρέφει άδειο σύνολο αν το ερώτημα είναι κενό

    result_set = set(inverted_index.get(terms[0], []))  # Ξεκινάμε με τον πρώτο όρο
    
    i = 1
    while i < len(terms):
        operator = terms[i]  # AND, OR, NOT
        
        # Επαλήθευση αν υπάρχει ο επόμενος όρος
        if i + 1 >= len(terms):
            print(f"Λάθος: Λείπει όρος μετά τον τελεστή '{operator}'")
            break
        
        next_term = terms[i + 1]
        next_docs = set(inverted_index.get(next_term, []))  # Έγγραφα για τον επόμενο όρο
        
        # Εκτέλεση της λειτουργίας
        if operator == "and":
            result_set = result_set.intersection(next_docs)
        elif operator == "or":
            result_set = result_set.union(next_docs)
        elif operator == "not":
            result_set = result_set.difference(next_docs)
        else:
            print(f"Λάθος: Άγνωστος τελεστής '{operator}'")
            break
        
        i += 2  # Επόμενος τελεστής ή όρος
    
    return result_set

# main Συνάρτηση
def main():
    # Φόρτωσε το ανεστραμμένο ευρετήριο
    filepath = "inverted_index.json"  
    inverted_index = load_inverted_index(filepath)

    print("Καλωσήρθατε στη Μηχανή Αναζήτησης!")
    print("Πληκτρολογήστε το ερώτημά σας χρησιμοποιώντας Boolean τελεστές (AND, OR, NOT) ή 'exit' για έξοδο.")
    
    while True:
        # Εισαγωγή ερωτήμ,ατος από τον χρήστη
        query = input("\nΕρώτημα: ").strip()
        
        # Έξοδος αν ο χρήστης πληκτρολογήσει "exit"
        if query.lower() == "exit":
            print("Έξοδος από τη Μηχανή Αναζήτησης!")
            break
        
        # Εκτέλεση Boolean Search
        results = boolean_search(query, inverted_index)
        
        # Εμφάνιση αποτελεσμάτων
        if results:
            print(f"Αποτελέσματα για το ερώτημα '{query}': {results}")
        else:
            print(f"Δεν βρέθηκαν αποτελέσματα για το ερώτημα '{query}'.")


if __name__ == "__main__":
    main()

β) Κατάταξη αποτελεσμάτων (Ranking):

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
import json
import numpy as np

# Συνάρτηση για τη φόρτωση εγγράφων από ένα αρχείο JSON
def load_documents(filepath):
    with open(filepath, "r", encoding="utf-8") as json_file:
        data = json.load(json_file)
        # Αν το JSON περιέχει λίστα, επιστρέφουμε τα περιεχόμενα του πεδίου "content"
        if isinstance(data, list):
            return [doc["content"] for doc in data if "content" in doc]
        # Αν το JSON περιέχει λεξικό (π.χ. το ανεστραμμένο ευρετήριο), επιστρέφουμε το λεξικό
        elif isinstance(data, dict):
            return data
        # Αν δεν είναι καμία από τις δύο μορφές, εμφανίζεται μήνυμα λάθους
        else:
            raise ValueError("Το αρχείο δεν έχει υποστηριζόμενη μορφή.")

# Boolean Search για αναζήτηση εγγράφων με βάση Boolean λογική (AND, OR, NOT)
def boolean_search(query, inverted_index):
    terms = query.lower().split()  # Διαχωρισμός του ερωτήματος σε λέξεις
    if not terms:  # Αν το ερώτημα είναι κενό, επιστρέφουμε κενό σύνολο
        return set()

    # Ξεκινάμε με τα έγγραφα που περιέχουν την πρώτη λέξη
    result_set = set(inverted_index.get(terms[0], []))
    i = 1
    while i < len(terms):  # Επεξεργασία των υπόλοιπων λέξεων και τελεστών
        operator = terms[i]  # Παίρνουμε τον Boolean τελεστή (AND, OR, NOT)
        if i + 1 >= len(terms):  # Αν δεν υπάρχει άλλη λέξη μετά τον τελεστή, σταματάμε
            break
        next_term = terms[i + 1]  # Επόμενη λέξη
        next_docs = set(inverted_index.get(next_term, []))  # Έγγραφα που περιέχουν τη λέξη
        # Εκτέλεση της αντίστοιχης Boolean λειτουργίας
        if operator == "and":
            result_set = result_set.intersection(next_docs)
        elif operator == "or":
            result_set = result_set.union(next_docs)
        elif operator == "not":
            result_set = result_set.difference(next_docs)
        i += 2  # Προχωράμε στον επόμενο τελεστή/λέξη
    return result_set

# TF-IDF Ranking για κατάταξη εγγράφων με βάση τη σχετικότητα (TF-IDF score)
def tfidf_ranking(query, documents):
    vectorizer = TfidfVectorizer()  # Δημιουργία του TF-IDF Vectorizer
    doc_vectors = vectorizer.fit_transform(documents)  # Υπολογισμός TF-IDF για τα έγγραφα
    query_vector = vectorizer.transform([query.lower()])  # Μετατροπή του ερωτήματος σε διάνυσμα
    scores = cosine_similarity(query_vector, doc_vectors).flatten()  # Υπολογισμός cosine similarity
    sorted_indices = np.argsort(scores)[::-1]  # Ταξινόμηση των εγγράφων με βάση τη βαθμολογία
    # Επιστροφή μόνο των εγγράφων με βαθμολογία > 0
    return [(idx, scores[idx]) for idx in sorted_indices if scores[idx] > 0.0]

# BM25 Ranking για κατάταξη εγγράφων με βάση το BM25
def bm25_ranking(query, documents):
    tokenized_docs = [doc.split() for doc in documents]  # Διαχωρισμός των εγγράφων σε λέξεις
    bm25 = BM25Okapi(tokenized_docs)  # Δημιουργία BM25 μοντέλου
    tokenized_query = query.split()  # Διαχωρισμός του ερωτήματος σε λέξεις
    scores = bm25.get_scores(tokenized_query)  # Υπολογισμός BM25 scores
    sorted_indices = np.argsort(scores)[::-1]  # Ταξινόμηση των εγγράφων με βάση τη βαθμολογία
    # Επιστροφή των εγγράφων με βαθμολογίες
    return [(idx, scores[idx]) for idx in sorted_indices if scores[idx] > 0.0]

# Κύρια συνάρτηση για την εκτέλεση της μηχανής αναζήτησης
def main():
    # Αρχεία εισόδου
    filepath_docs = "articles.json"  # Αρχείο με τα έγγραφα
    filepath_index = "inverted_index.json"  # Αρχείο με το ανεστραμμένο ευρετήριο
    documents = load_documents(filepath_docs)  # Φόρτωση των εγγράφων
    inverted_index = load_documents(filepath_index)  # Φόρτωση του ανεστραμμένου ευρετηρίου

    # Εμφάνιση επιλογών στον χρήστη
    print("Επιλέξτε αλγόριθμο ανάκτησης:")
    print("1. Boolean Retrieval")
    print("2. TF-IDF Ranking")
    print("3. BM25 Ranking")
    print("Πληκτρολογήστε 'exit' για έξοδο.")

    while True:
        choice = input("\nΕπιλογή: ").strip()  # Είσοδος επιλογής από τον χρήστη
        if choice.lower() == "exit":  # Έξοδος από το πρόγραμμα
            print("Έξοδος από τη Μηχανή Αναζήτησης!")
            break

        query = input("Ερώτημα: ").strip()  # Είσοδος ερωτήματος
        if not query:  # Αν το ερώτημα είναι κενό
            print("Το ερώτημα δεν μπορεί να είναι κενό.")
            continue

        if choice == "1":  # Boolean Retrieval
            results = boolean_search(query, inverted_index)
            if results:
                print("\nBoolean Results:")
                for idx in results:
                    if idx < len(documents):  # Έλεγχος αν το έγγραφο υπάρχει
                        print(f"Έγγραφο {idx}")
                    else:
                        print(f"Το έγγραφο {idx} δεν υπάρχει στο articles.json.")
            else:
                print("Δεν βρέθηκαν αποτελέσματα.")

        elif choice == "2":  # TF-IDF Ranking
            results = tfidf_ranking(query, documents)
            print("\nTF-IDF Results:")
            for idx, score in results:
                if idx < len(documents):  # Έλεγχος αν το έγγραφο υπάρχει
                    print(f"Έγγραφο {idx} (Σκορ: {score:.4f})")
                else:
                    print(f"Το έγγραφο {idx} δεν υπάρχει στο articles.json.")

        elif choice == "3":  # BM25 Ranking
            results = bm25_ranking(query, documents)
            print("\nBM25 Results:")
            for idx, score in results:
                if idx < len(documents):  # Έλεγχος αν το έγγραφο υπάρχει
                    print(f"Έγγραφο {idx} (Σκορ: {score:.4f})")
                else:
                    print(f"Το έγγραφο {idx} δεν υπάρχει στο articles.json.")

        else:  # Μη έγκυρη επιλογή
            print("Μη έγκυρη επιλογή. Παρακαλώ προσπαθήστε ξανά.")

# Εκκίνηση προγράμματος
if __name__ == "__main__":
    main()